Consider GPT-2 XL, which has the following configuration:
-  vocab_size: 50,257
-   context_length: 1,024
-   num_layers: 48
-   d_model: 1,600
-   num_heads: 25
-   d_ff: 6,400

Suppose we constructed our model using this configuration. How many trainable parameters would our model have? Assuming each parameter is represented using singl-precision floating point, how much memory is required to just load this model?

In [137]:
vocab_size = 50257
context_length = 1024
num_layers = 48
d_model = 1600
num_heads = 25
d_ff = 6400

token_embeddings = vocab_size * d_model
position_embeddings = context_length * d_model
embd_layer = token_embeddings + position_embeddings

q_k_v_o = 4 * d_model * d_model
ffn = 2 * d_model * d_ff
transformer_blocks = num_layers * (q_k_v_o + ffn)

lm_head = d_model * vocab_size

total_params = embd_layer + transformer_blocks + lm_head
print(f"Total trainable params {total_params/10**9:0.2f}B")
bytes_float32 = total_params * 4
mem_fp32 = bytes_float32 / 1024 ** 3
bytes_bf16 = total_params * 2
mem_bf16 = bytes_bf16 / 1024 ** 3
print(f"Memory fp32: {mem_fp32:,.2f} GiB")
print(f"Memory bf16: {mem_bf16:,.2f} GiB")

Total trainable params 1.64B
Memory fp32: 6.10 GiB
Memory bf16: 3.05 GiB


Identify the matrix multiplies required to complete a forward pass of our GPT-2 XL-shaped model.
How many FLOPs do these matrix multiplies require in total? Assume that our input
sequence has `context_length` tokens

In [136]:
def get_overall_flops(
    vocab_size,
    context_length,
    num_layers,
    d_model,
    num_heads,
    d_ff,
    b = 1
):
    F_forward_pass = get_forward_flops_TransformerLM(
        vocab_size, context_length, num_layers, d_model, num_heads, d_ff, b
    )
    F_backward_pass = get_backward_flops_TransformerLM(
        F_forward_pass
    )
    return F_forward_pass + F_backward_pass

def get_forward_flops_TransformerLM(
    vocab_size,
    context_length,
    num_layers,
    d_model,
    num_heads,
    d_ff,
    b = 1
):
    F_transformer_blocks = get_flops_TransformerBlock(
        b, context_length, d_model, num_heads, d_ff
    ) * num_layers
    F_ln_final = get_flops_RMSNorm(b, context_length, d_model)
    F_lm_head = get_flops_lm_head(b, context_length, d_model, vocab_size)
    return F_transformer_blocks + F_ln_final + F_lm_head

def get_backward_flops_TransformerLM(
    F_forward_pass
):
    F_backward_pass = 2 * F_forward_pass
    return F_backward_pass


def get_flops_lm_head(b, context_length, d_model, vocab_size):
    return 2 * b * context_length * d_model * vocab_size

def get_flops_TransformerBlock(b, context_length, d_model, num_heads, d_ff):
    F_rmsnorm_1 = get_flops_RMSNorm(b, context_length, d_model)
    F_mha = get_flops_MultiHeadAttention(b, context_length, d_model, num_heads)
    F_rmsnorm_2 = F_rmsnorm_1
    F_ffn = get_flops_FFN(b, context_length, d_model, d_ff)
    return F_rmsnorm_1 + F_mha + F_rmsnorm_2 + F_ffn

def get_flops_RMSNorm(b, context_length, d_model):
    return (3*d_model + 2) * b * context_length

def get_flops_MultiHeadAttention(b, context_length, d_model, num_heads):
    F_QKV_linear_proj = get_qkv_proj(b, context_length, d_model)
    F_attention = get_flops_atten_scores(b, context_length, d_model)
    F_softmax = get_flops_Softmax(b, context_length, num_heads)
    F_weighted = get_flops_atten_scores(b, context_length, d_model)
    F_out_proj = get_flops_output_proj(b, context_length, d_model)
    return F_QKV_linear_proj + F_attention + F_softmax + F_weighted + F_out_proj

def get_qkv_proj(b, context_length, d_model):
    return 3 * (2 * b * context_length * (d_model**2))

def get_flops_atten_scores(b, context_length, d_model):
    return 2 * b * (context_length**2) * d_model

def get_flops_Softmax(b, context_length, num_heads):
    return 5 * b * (context_length**2) * num_heads

def get_flops_output_proj(b, context_length, d_model):
    return 2 * b * context_length * d_model * d_model

def get_flops_FFN(b, context_length, d_model, d_ff):
    F_w1 = 2 * b * context_length * d_model * d_ff
    F_gelu = get_flops_GELU(b, context_length, d_ff)
    F_w2 = F_w1
    return F_w1 + F_gelu + F_w2

def get_flops_GELU(b, context_length, d_ff):
    return 5 * b * context_length * d_ff

def get_table_all_flops(
    vocab_size,
    context_length,
    num_layers,
    d_model,
    num_heads,
    d_ff,
    b = 1
):
    all_model_flops = {
        "Transformer_block": {
            "RMSNorm_1": get_flops_RMSNorm(b, context_length, d_model),
            "MultiHeadAttentio": {
                "QKV_proj": get_qkv_proj(b, context_length, d_model),
                "Attention_scores": get_flops_atten_scores(b, context_length, d_model),
                "Softmax": get_flops_Softmax(b, context_length, num_heads),
                "Attention_weighted": get_flops_atten_scores(b, context_length, d_model),
                "O_proj": get_flops_output_proj(b, context_length, d_model),
                "Overall_MultiHeadAttentio": get_flops_MultiHeadAttention(b, context_length, d_model, num_heads)
            },
            "RMSNorm_2": get_flops_RMSNorm(b, context_length, d_model),
            "FFN": get_flops_FFN(b, context_length, d_model, d_ff),
            "Overall_Transformer_block": get_flops_TransformerBlock(
                b, context_length, d_model, num_heads, d_ff
            ) * num_layers
        },
        "RMSNorm_final": get_flops_RMSNorm(b, context_length, d_model),
        "LM_head": get_flops_lm_head(b, context_length, d_model, vocab_size),
        "forward_pass_TransformerLM": get_forward_flops_TransformerLM(
            vocab_size, context_length, num_layers, d_model, num_heads, d_ff, b
        ),
        "backward_pass_TransformerLM": get_backward_flops_TransformerLM(
            get_forward_flops_TransformerLM(
                vocab_size, context_length, num_layers, d_model, num_heads, d_ff, b)
        ),
        "Overall_TransformerLM": get_overall_flops(
            vocab_size, context_length, num_layers, d_model, num_heads, d_ff, b
        )
    }
    return all_model_flops

In [138]:
import pandas as pd

# --- your FLOP-computing helpers remain unchanged here ------------------ #
# (get_flops_RMSNorm, get_flops_MultiHeadAttention, … get_table_all_flops)
# ----------------------------------------------------------------------- #

def explode_to_tuples(d, prefix=()):
    """Yield (path_tuple, value) pairs from a nested dict."""
    for k, v in d.items():
        path = prefix + (k,)
        if isinstance(v, dict):
            yield from explode_to_tuples(v, path)
        else:
            yield path, v

def flops_table_multiindex(
    vocab_size, context_len, n_layers, d_model, n_heads, d_ff, b=1
):
    # 1. get the nested FLOPs dict
    nested = get_table_all_flops(
        vocab_size, context_len, n_layers, d_model, n_heads, d_ff, b
    )
    overll_flops = nested['Overall_TransformerLM']
    # 2. explode to (tuple, value) pairs
    tuples, vals = zip(*explode_to_tuples(nested))

    # 3. make all tuples the same length
    max_depth = max(len(t) for t in tuples)
    padded = [t + ("",) * (max_depth - len(t)) for t in tuples]

    # 4. build the MultiIndex DataFrame
    names = [f"Level-{i+1}" for i in range(max_depth)]
    mi = pd.MultiIndex.from_tuples(padded, names=names)
    df = pd.DataFrame({"FLOPs": vals}, index=mi)

    # 5. optional pretty formatting
    df["FLOPs_fmt"] = df["FLOPs"].astype(float).map("{:.2e}".format)
    df["FLOPs_proportion"] = df["FLOPs"] / overll_flops
    return df


gpt2_xl = flops_table_multiindex(
    vocab_size=50257,
    context_len=1024,
    n_layers=48,
    d_model=1600,
    n_heads=25,
    d_ff=6400,
)

gpt2_xl

FLOPs  \
Level-1                     Level-2                   Level-3                                     
Transformer_block           RMSNorm_1                                                   4917248   
                            MultiHeadAttentio         QKV_proj                      15728640000   
                                                      Attention_scores               3355443200   
                                                      Softmax                         131072000   
                                                      Attention_weighted             3355443200   
                                                      O_proj                         5242880000   
                                                      Overall_MultiHeadAttentio     27813478400   
                            RMSNorm_2                                                   4917248   
                            FFN                                                     41975808000   
                            Overall_Transformer_block                             3350357803008   
RMSNorm_final                                                                           4917248   
LM_head                                                                            164682137600   
forward_pass_TransformerLM                                                        3515044857856   
backward_pass_TransformerLM                                                       7030089715712   
Overall_TransformerLM                                                            10545134573568   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            4.92e+06   
                            MultiHeadAttentio         QKV_proj                   1.57e+10   
                                                      Attention_scores           3.36e+09   
                                                      Softmax                    1.31e+08   
                                                      Attention_weighted         3.36e+09   
                                                      O_proj                     5.24e+09   
                                                      Overall_MultiHeadAttentio  2.78e+10   
                            RMSNorm_2                                            4.92e+06   
                            FFN                                                  4.20e+10   
                            Overall_Transformer_block                            3.35e+12   
RMSNorm_final                                                                    4.92e+06   
LM_head                                                                          1.65e+11   
forward_pass_TransformerLM                                                       3.52e+12   
backward_pass_TransformerLM                                                      7.03e+12   
Overall_TransformerLM                                                            1.05e+13   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                4.663049e-07  
                            MultiHeadAttentio         QKV_proj                       1.491554e-03  
                                                      Attention_scores               3.181982e-04  
                                                      Softmax                        1.242962e-05  
                                                      Attention_weighted             3.181982e-04  
                                                      O_proj                         4.971847e-04  
                        

In [151]:
a = 6 * total_params * 50e3
a = 6 * num_layers * d_model*d_model * context_length + 12 * num_layers * context_length * context_length  * d_model
a / 10545134573568

0.16323569411002548

In [130]:
gpt2_small = flops_table_multiindex(
    vocab_size=50257,
    context_len=1024,
    n_layers=12,
    d_model=768,
    n_heads=12,
    d_ff=6400,
)

gpt2_small

FLOPs  \
Level-1                     Level-2                   Level-3                                    
Transformer_block           RMSNorm_1                                                  2361344   
                            MultiHeadAttentio         QKV_proj                      3623878656   
                                                      Attention_scores              1610612736   
                                                      Softmax                         62914560   
                                                      Attention_weighted            1610612736   
                                                      O_proj                        1207959552   
                                                      Overall_MultiHeadAttentio     8115978240   
                            RMSNorm_2                                                  2361344   
                            FFN                                                    20165427200   
                            Overall_Transformer_block                             339433537536   
RMSNorm_final                                                                          2361344   
LM_head                                                                            79047426048   
forward_pass_TransformerLM                                                        418483324928   
backward_pass_TransformerLM                                                       836966649856   
Overall_TransformerLM                                                            1255449974784   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            2.36e+06   
                            MultiHeadAttentio         QKV_proj                   3.62e+09   
                                                      Attention_scores           1.61e+09   
                                                      Softmax                    6.29e+07   
                                                      Attention_weighted         1.61e+09   
                                                      O_proj                     1.21e+09   
                                                      Overall_MultiHeadAttentio  8.12e+09   
                            RMSNorm_2                                            2.36e+06   
                            FFN                                                  2.02e+10   
                            Overall_Transformer_block                            3.39e+11   
RMSNorm_final                                                                    2.36e+06   
LM_head                                                                          7.90e+10   
forward_pass_TransformerLM                                                       4.18e+11   
backward_pass_TransformerLM                                                      8.37e+11   
Overall_TransformerLM                                                            1.26e+12   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                    0.000002  
                            MultiHeadAttentio         QKV_proj                           0.002887  
                                                      Attention_scores                   0.001283  
                                                      Softmax                            0.000050  
                                                      Attention_weighted                 0.001283  
                                                      O_proj                             0.000962  
                                        

In [131]:
gpt2_medium = flops_table_multiindex(
    vocab_size=50257,
    context_len=1024,
    n_layers=24,
    d_model=1024,
    n_heads=16,
    d_ff=6400,
)

gpt2_medium

FLOPs  \
Level-1                     Level-2                   Level-3                                    
Transformer_block           RMSNorm_1                                                  3147776   
                            MultiHeadAttentio         QKV_proj                      6442450944   
                                                      Attention_scores              2147483648   
                                                      Softmax                         83886080   
                                                      Attention_weighted            2147483648   
                                                      O_proj                        2147483648   
                                                      Overall_MultiHeadAttentio    12968787968   
                            RMSNorm_2                                                  3147776   
                            FFN                                                    26876313600   
                            Overall_Transformer_block                             956433530880   
RMSNorm_final                                                                          3147776   
LM_head                                                                           105396568064   
forward_pass_TransformerLM                                                       1061833246720   
backward_pass_TransformerLM                                                      2123666493440   
Overall_TransformerLM                                                            3185499740160   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            3.15e+06   
                            MultiHeadAttentio         QKV_proj                   6.44e+09   
                                                      Attention_scores           2.15e+09   
                                                      Softmax                    8.39e+07   
                                                      Attention_weighted         2.15e+09   
                                                      O_proj                     2.15e+09   
                                                      Overall_MultiHeadAttentio  1.30e+10   
                            RMSNorm_2                                            3.15e+06   
                            FFN                                                  2.69e+10   
                            Overall_Transformer_block                            9.56e+11   
RMSNorm_final                                                                    3.15e+06   
LM_head                                                                          1.05e+11   
forward_pass_TransformerLM                                                       1.06e+12   
backward_pass_TransformerLM                                                      2.12e+12   
Overall_TransformerLM                                                            3.19e+12   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                9.881577e-07  
                            MultiHeadAttentio         QKV_proj                       2.022430e-03  
                                                      Attention_scores               6.741434e-04  
                                                      Softmax                        2.633373e-05  
                                                      Attention_weighted             6.741434e-04  
                                                      O_proj                         6.741434e-04  
                                        

In [132]:
gpt2_large = flops_table_multiindex(
    vocab_size=50257,
    context_len=1024,
    n_layers=36,
    d_model=1280,
    n_heads=20,
    d_ff=6400,
)

gpt2_large

FLOPs  \
Level-1                     Level-2                   Level-3                                    
Transformer_block           RMSNorm_1                                                  3934208   
                            MultiHeadAttentio         QKV_proj                     10066329600   
                                                      Attention_scores              2684354560   
                                                      Softmax                        104857600   
                                                      Attention_weighted            2684354560   
                                                      O_proj                        3355443200   
                                                      Overall_MultiHeadAttentio    18895339520   
                            RMSNorm_2                                                  3934208   
                            FFN                                                    33587200000   
                            Overall_Transformer_block                            1889654685696   
RMSNorm_final                                                                          3934208   
LM_head                                                                           131745710080   
forward_pass_TransformerLM                                                       2021404329984   
backward_pass_TransformerLM                                                      4042808659968   
Overall_TransformerLM                                                            6064212989952   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            3.93e+06   
                            MultiHeadAttentio         QKV_proj                   1.01e+10   
                                                      Attention_scores           2.68e+09   
                                                      Softmax                    1.05e+08   
                                                      Attention_weighted         2.68e+09   
                                                      O_proj                     3.36e+09   
                                                      Overall_MultiHeadAttentio  1.89e+10   
                            RMSNorm_2                                            3.93e+06   
                            FFN                                                  3.36e+10   
                            Overall_Transformer_block                            1.89e+12   
RMSNorm_final                                                                    3.93e+06   
LM_head                                                                          1.32e+11   
forward_pass_TransformerLM                                                       2.02e+12   
backward_pass_TransformerLM                                                      4.04e+12   
Overall_TransformerLM                                                            6.06e+12   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                6.487582e-07  
                            MultiHeadAttentio         QKV_proj                       1.659956e-03  
                                                      Attention_scores               4.426551e-04  
                                                      Softmax                        1.729121e-05  
                                                      Attention_weighted             4.426551e-04  
                                                      O_proj                         5.533188e-04  
                                        

In [134]:
gpt2_xl = flops_table_multiindex(
    vocab_size=50257,
    context_len=1024,
    n_layers=48,
    d_model=1600,
    n_heads=25,
    d_ff=6400,
)

gpt2_xl

FLOPs  \
Level-1                     Level-2                   Level-3                                     
Transformer_block           RMSNorm_1                                                   4917248   
                            MultiHeadAttentio         QKV_proj                      15728640000   
                                                      Attention_scores               3355443200   
                                                      Softmax                         131072000   
                                                      Attention_weighted             3355443200   
                                                      O_proj                         5242880000   
                                                      Overall_MultiHeadAttentio     27813478400   
                            RMSNorm_2                                                   4917248   
                            FFN                                                     41975808000   
                            Overall_Transformer_block                             3350357803008   
RMSNorm_final                                                                           4917248   
LM_head                                                                            164682137600   
forward_pass_TransformerLM                                                        3515044857856   
backward_pass_TransformerLM                                                       7030089715712   
Overall_TransformerLM                                                            10545134573568   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            4.92e+06   
                            MultiHeadAttentio         QKV_proj                   1.57e+10   
                                                      Attention_scores           3.36e+09   
                                                      Softmax                    1.31e+08   
                                                      Attention_weighted         3.36e+09   
                                                      O_proj                     5.24e+09   
                                                      Overall_MultiHeadAttentio  2.78e+10   
                            RMSNorm_2                                            4.92e+06   
                            FFN                                                  4.20e+10   
                            Overall_Transformer_block                            3.35e+12   
RMSNorm_final                                                                    4.92e+06   
LM_head                                                                          1.65e+11   
forward_pass_TransformerLM                                                       3.52e+12   
backward_pass_TransformerLM                                                      7.03e+12   
Overall_TransformerLM                                                            1.05e+13   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                4.663049e-07  
                            MultiHeadAttentio         QKV_proj                       1.491554e-03  
                                                      Attention_scores               3.181982e-04  
                                                      Softmax                        1.242962e-05  
                                                      Attention_weighted             3.181982e-04  
                                                      O_proj                         4.971847e-04  
                        

In [135]:
gpt2_xl = flops_table_multiindex(
    vocab_size=50257,
    context_len=16384,
    n_layers=48,
    d_model=1600,
    n_heads=25,
    d_ff=6400,
)

gpt2_xl

FLOPs  \
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                   78675968   
                            MultiHeadAttentio         QKV_proj                      251658240000   
                                                      Attention_scores              858993459200   
                                                      Softmax                        33554432000   
                                                      Attention_weighted            858993459200   
                                                      O_proj                         83886080000   
                                                      Overall_MultiHeadAttentio    2087085670400   
                            RMSNorm_2                                                   78675968   
                            FFN                                                     671612928000   
                            Overall_Transformer_block                            132425085616128   
RMSNorm_final                                                                           78675968   
LM_head                                                                            2634914201600   
forward_pass_TransformerLM                                                       135060078493696   
backward_pass_TransformerLM                                                      270120156987392   
Overall_TransformerLM                                                            405180235481088   

                                                                                FLOPs_fmt  \
Level-1                     Level-2                   Level-3                               
Transformer_block           RMSNorm_1                                            7.87e+07   
                            MultiHeadAttentio         QKV_proj                   2.52e+11   
                                                      Attention_scores           8.59e+11   
                                                      Softmax                    3.36e+10   
                                                      Attention_weighted         8.59e+11   
                                                      O_proj                     8.39e+10   
                                                      Overall_MultiHeadAttentio  2.09e+12   
                            RMSNorm_2                                            7.87e+07   
                            FFN                                                  6.72e+11   
                            Overall_Transformer_block                            1.32e+14   
RMSNorm_final                                                                    7.87e+07   
LM_head                                                                          2.63e+12   
forward_pass_TransformerLM                                                       1.35e+14   
backward_pass_TransformerLM                                                      2.70e+14   
Overall_TransformerLM                                                            4.05e+14   

                                                                                 FLOPs_proportion  
Level-1                     Level-2                   Level-3                                      
Transformer_block           RMSNorm_1                                                1.941752e-07  
                            MultiHeadAttentio         QKV_proj                       6.211020e-04  
                                                      Attention_scores               2.120028e-03  
                                                      Softmax                        8.281360e-05  
                                                      Attention_weighted             2.120028e-03  
                                                      O_proj                         2.070340e-04  
        